In [1]:
import random
import itertools

import numpy as np
import qiskit.quantum_info as qi

from tqdm import tqdm
from codes import swapper, get_marginals, physical_imposition_operator, get_state_of_rank

In [2]:
# Define the parameters of the data that's going to be generated for training stage

dimension   = 2     # Dimension of each qudits, for qubits is 2.
n_qubits    = 4     # Number of qubits of the global state
n_marginal  = 3     # Number of marginal that we know
n_matrices  = 5000  # Number of matrices to be generated per rank

In [3]:
# To define the data is necessary to define a swapper matrix and label the differents marginal

matrix_size = dimension ** n_qubits
subs_dims   = [dimension for _ in range(n_qubits)]
swapper_d   = swapper(dimension)
labels_mg   = list(itertools.combinations(range(n_qubits), r = n_marginal))

In [4]:
# We generates the data to be used in training stage

data = []

for rank in range(7,matrix_size + 1):
     for iter in tqdm(range(n_matrices), desc = "Generating matrices of rank "+str(rank)):
          # We generates a mixed state and, then, get the marginals related with the labels_mg subsystems
          rho_noiseless = get_state_of_rank(random.randint(rank, matrix_size), dimension, n_qubits)
          marginals_in  = get_marginals(rho_noiseless, dimension, n_qubits, labels_mg)
          
          # We impose marginals in a random initial state. The output, named 'rho_noisy', isn't necessarily positive semidefinite
          rho_noisy = physical_imposition_operator(
               dimension, 
               n_qubits, 
               qi.random_density_matrix(dims=subs_dims).data, 
               marginals_in,
               swapper_d
          )

          data.append([np.stack((rho_noisy.real.astype(np.float32), rho_noisy.imag.astype(np.float32))),
                    np.stack((rho_noiseless.real.astype(np.float32), rho_noiseless.imag.astype(np.float32)))])

print("Saving data ...")
np.savez_compressed(f"data/data_{n_qubits}body_{n_marginal}parts_d{dimension}", data)
print("Data saved. Process finished!")

Generating matrices of rank 16: 100%|██████████| 5000/5000 [00:06<00:00, 788.03it/s]


Saving data ...
Data saved. Process finished!
